[![notebook shield](https://img.shields.io/static/v1?label=&message=Notebook&color=blue&style=for-the-badge&logo=googlecolab&link=https://colab.research.google.com/github/ArthurFDLR/whisper-youtube/blob/main/whisper_youtube.ipynb)](https://colab.research.google.com/github/Majdoddin/nlp/blob/main/Pyannote_plays_and_Whisper_rhymes_v_2_0.ipynb)
[![repository shield](https://img.shields.io/static/v1?label=&message=Repository&color=blue&style=for-the-badge&logo=github&link=https://github.com/openai/whisper)](https://github.com/majdoddin/nlp)

# Whisper's transcription plus Pyannote's Diarization 

Andrej Karpathy [suggested](https://twitter.com/karpathy/status/1574476200801538048?s=20&t=s5IMMXOYjBI6-91dib6w8g) training a classifier on top of  OpenAI [Whisper](https://openai.com/blog/whisper/) model features to identify the speaker, so we can visualize the speaker in the transcript. But, as [pointed out](https://twitter.com/tarantulae/status/1574493613362388992?s=20&t=s5IMMXOYjBI6-91dib6w8g) by Christian Perone, it seems that features from whisper wouldn't be that great for speaker recognition as its main objective is basically to ignore speaker differences.

In the following, I use [**`pyannote-audio`**](https://github.com/pyannote/pyannote-audio), a speaker diarization toolkit by Hervé Bredin, to identify the speakers, and then match it with the transcriptions of Whispr, linked to the video. The input can be YouTube or an video/audio file (also on Google Drive). I try it on a part of an [interview](https://youtu.be/NSp2fEQ6wyA) with Freeman Dyson. Check the result [**here**](https://majdoddin.github.io/dyson.html).

To make it easier to match the transcriptions to diarizations by speaker change, Sarah Kaiser [suggested](https://github.com/openai/whisper/discussions/264#discussioncomment-3825375) runnnig the pyannote.audio first and  then just running whisper on the split-by-speaker chunks. 
For sake of performance (and transcription quality?), we attach the audio segements into a single audio file with a silent spacer as a seperator, and run whisper on it. Enjoy it!

(For sake of performance , I also tried attaching the audio segements into a single audio file with a silent -or beep- spacer as a seperator, and run whisper on it see it on [colab](https://colab.research.google.com/drive/1HuvcY4tkTHPDzcwyVH77LCh_m8tP-Qet?usp=sharing). It [works](https://majdoddin.github.io/lexicap.html) on some audio, and fails on some (Dyson's Interview). The problem is, whisper does not reliably make a timestap on a spacer. See the discussions [#139](https://github.com/openai/whisper/discussions/139) and [#29](https://github.com/openai/whisper/discussions/29))

The Markdown form used below is from [@ArthurFDLR](https://github.com/ArthurFDLR/whisper-youtube/).   

# Preparing the audio file

**Optional:** Mount Google Drive



In [ ]:
from google.colab import drive
from pathlib import Path

drive_mount_path = Path("/content/drive")
drive.mount(str(drive_mount_path))
drive_mount_path /= "MyDrive"

Mounted at /content/drive


In [ ]:
#@markdown Enter the URL of the YouTube video, or the path to the video/audio file you want to transcribe, give the output path, and run the cell. HTML file is generated only for YouTube videos

Source = 'Youtube' #@param ['Youtube', 'File (Google Drive)']
#@markdown ---
#@markdown #### **Youtube video**
video_url = "https://youtu.be/NSp2fEQ6wyA" #@param {type:"string"}
#store_audio = True #@param {type:"boolean"}
#@markdown ---
#@markdown #### **Google Drive video or audio (mp4, wav)**
video_path = "/content/drive/MyDrive/Colab Notebooks/PyannoteWhisper/dyson.mp4" #@param {type:"string"}
#@markdown ---
output_path = "/content/drive/MyDrive/Colab Notebooks/PyannoteWhisper/" #@param {type:"string"}
output_path = str(Path(output_path))
#@markdown ---
#@markdown **Run this cell again if you change the video.**


In [ ]:
Path(output_path).mkdir(parents=True, exist_ok=True)
%cd {output_path}
video_title = ""
video_id = ""

/content/drive/MyDrive/Colab Notebooks/PyannoteWhisper


## From YouTube

 Installing [`yt-dlp`](https://github.com/yt-dlp/yt-dlp) and downloading the [video](https://youtu.be/NSp2fEQ6wyA) from youtube.

In [ ]:
if Source == "Youtube":
  !pip install -U yt-dlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 4.3 MB/s 
     |████████████████████████████████| 2.3 MB 53.5 MB/s 
     |████████████████████████████████| 357 kB 57.2 MB/s 
     |████████████████████████████████| 193 kB 94.8 MB/s 
     |████████████████████████████████| 106 kB 87.9 MB/s 


Custom build of `ffmpeg` as [recommended](https://github.com/yt-dlp/yt-dlp#strongly-recommended) by `yt-dlp`.

In [ ]:
if Source == "Youtube":
  !wget -O - -q  https://github.com/yt-dlp/FFmpeg-Builds/releases/download/latest/ffmpeg-master-latest-linux64-gpl.tar.xz | xz -qdc| tar -x

In [ ]:
#Getting video info
if Source == "Youtube":
  from yt_dlp import YoutubeDL
  with YoutubeDL() as ydl: 
    info_dict = ydl.extract_info(video_url, download=False)
    video_title = info_dict.get('title', None)
    video_id = info_dict.get('id', None)
    print("Title: " + video_title) # <= Here, you got the video title


[youtube] NSp2fEQ6wyA: Downloading webpage
[youtube] NSp2fEQ6wyA: Downloading android player API JSON
Title: Freeman Dyson - Pure mathematics at Cambridge: the influence of Besicovitch (23/157)


Downloading the audio from YouTube.

In [ ]:
if Source == "Youtube":
  !yt-dlp -xv --ffmpeg-location ffmpeg-master-latest-linux64-gpl/bin --audio-format wav  -o "{str(output_path) + '/'}input.wav" -- {video_url}

## or from File (Google Drive)

In [ ]:
if Source == 'File (Google Drive)':
    !ffmpeg -i {repr(video_path)} -vn -acodec pcm_s16le -ar 16000 -ac 1 input.wav  

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

## Prepending a spacer

`pyannote.audio` seems to miss the first 0.5 seconds of the audio, and, therefore, we prepend a spcacer.

In [ ]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pydub import AudioSegment

spacermilli = 2000
spacer = AudioSegment.silent(duration=spacermilli)


audio = AudioSegment.from_wav("input.wav") 

audio = spacer.append(audio, crossfade=0)

audio.export('input_prep.wav', format='wav')

<_io.BufferedRandom name='input_prep.wav'>

# Pyannote's Diarization

[`pyannote.audio`](https://github.com/pyannote/pyannote-audio) is an open-source toolkit written in Python for **speaker diarization**. 

Based on [`PyTorch`](https://pytorch.org) machine learning framework, it provides a set of trainable end-to-end neural building blocks that can be combined and jointly optimized to build speaker diarization pipelines. 

`pyannote.audio` also comes with pretrained [models](https://huggingface.co/models?other=pyannote-audio-model) and [pipelines](https://huggingface.co/models?other=pyannote-audio-pipeline) covering a wide range of domains for voice activity detection, speaker segmentation, overlapped speech detection, speaker embedding reaching state-of-the-art performance for most of them. 

Installing `pyannote.audio`.

In [ ]:
!pip install   pyannote.audio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 390 kB 4.1 MB/s 
     |████████████████████████████████| 585 kB 45.5 MB/s 
     |████████████████████████████████| 163 kB 80.6 MB/s 
     |████████████████████████████████| 111 kB 13.2 MB/s 
     |████████████████████████████████| 47 kB 4.9 MB/s 
     |████████████████████████████████| 217 kB 67.7 MB/s 
     |████████████████████████████████| 60 kB 8.6 MB/s 
     |████████████████████████████████| 498 kB 79.5 MB/s 
     |████████████████████████████████| 51 kB 196 kB/s 
     |████████████████████████████████| 79 kB 8.6 MB/s 
     |████████████████████████████████| 41 kB 494 kB/s 
     |████████████████████████████████| 529 kB 71.1 MB/s 
     |████████████████████████████████| 117 kB 61.8 MB/s 
     |████████████████████████████████| 130 kB 66.9 MB/s 
     |████████████████████████████████| 348 kB 47.5 MB/s 
     |████████████████████████████████| 20

**Important:** To load the speaker diarization pipeline, 

* accept the user conditions on both [hf.co/pyannote/speaker-diarization](https://hf.co/pyannote/speaker-diarization) and [hf.co/pyannote/segmentation](https://huggingface.co/pyannote/segmentation).
* login using `notebook_login` below

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
from pyannote.audio import Pipeline

pipeline = Pipeline.from_pretrained('pyannote/speaker-diarization', use_auth_token=True)

Downloading:   0%|          | 0.00/500 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/318 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129k [00:00<?, ?B/s]

Running pyannote.audio to generate the diarizations.

In [ ]:
DEMO_FILE = {'uri': 'blabla', 'audio': 'input_prep.wav'}
dz = pipeline(DEMO_FILE)  

with open("diarization.txt", "w") as text_file:
    text_file.write(str(dz))

In [ ]:
print(*list(dz.itertracks(yield_label = True))[:10], sep="\n")

(<Segment(2.03344, 5.59406)>, 'A', 'SPEAKER_00')
(<Segment(6.80906, 7.11281)>, 'B', 'SPEAKER_00')
(<Segment(7.75406, 19.9378)>, 'C', 'SPEAKER_00')
(<Segment(20.8997, 24.0891)>, 'D', 'SPEAKER_00')
(<Segment(25.1016, 34.1297)>, 'E', 'SPEAKER_00')
(<Segment(34.8891, 37.8928)>, 'F', 'SPEAKER_00')
(<Segment(38.8547, 40.5084)>, 'G', 'SPEAKER_00')
(<Segment(41.8078, 46.0941)>, 'H', 'SPEAKER_00')
(<Segment(47.1234, 49.7053)>, 'I', 'SPEAKER_00')
(<Segment(50.2959, 51.6291)>, 'J', 'SPEAKER_00')


# Preparing audio files according to the diarization

In [ ]:
def millisec(timeStr):
  spl = timeStr.split(":")
  s = (int)((int(spl[0]) * 60 * 60 + int(spl[1]) * 60 + float(spl[2]) )* 1000)
  return s

Grouping the diarization segments according to the speaker.

In [ ]:
import re
dzs = open('diarization.txt').read().splitlines()

groups = []
g = []
lastend = 0

for d in dzs:   
  if g and (g[0].split()[-1] != d.split()[-1]):      #same speaker
    groups.append(g)
    g = []
  
  g.append(d)
  
  end = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=d)[1]
  end = millisec(end)
  if (lastend > end):       #segment engulfed by a previous segment
    groups.append(g)
    g = [] 
  else:
    lastend = end
if g:
  groups.append(g)
print(*groups, sep='\n')

['[ 00:00:02.033 -->  00:00:05.594] A SPEAKER_00', '[ 00:00:06.809 -->  00:00:07.112] B SPEAKER_00', '[ 00:00:07.754 -->  00:00:19.937] C SPEAKER_00', '[ 00:00:20.899 -->  00:00:24.089] D SPEAKER_00', '[ 00:00:25.101 -->  00:00:34.129] E SPEAKER_00', '[ 00:00:34.889 -->  00:00:37.892] F SPEAKER_00', '[ 00:00:38.854 -->  00:00:40.508] G SPEAKER_00', '[ 00:00:41.807 -->  00:00:46.094] H SPEAKER_00', '[ 00:00:47.123 -->  00:00:49.705] I SPEAKER_00', '[ 00:00:50.295 -->  00:00:51.629] J SPEAKER_00', '[ 00:00:52.304 -->  00:00:53.586] K SPEAKER_00', '[ 00:00:55.274 -->  00:00:56.370] L SPEAKER_00', '[ 00:00:58.024 -->  00:01:00.252] M SPEAKER_00', '[ 00:01:01.332 -->  00:01:10.377] N SPEAKER_00', '[ 00:01:11.440 -->  00:01:15.692] O SPEAKER_00', '[ 00:01:16.604 -->  00:01:23.455] P SPEAKER_00', '[ 00:01:24.315 -->  00:01:31.437] Q SPEAKER_00', '[ 00:01:32.669 -->  00:01:33.225] R SPEAKER_00', '[ 00:01:36.094 -->  00:01:37.090] S SPEAKER_00', '[ 00:01:39.435 -->  00:01:40.853] T SPEAKER_00',

Save the audio part corresponding to each diarization group.

In [ ]:
audio = AudioSegment.from_wav("input_prep.wav")
gidx = -1
for g in groups:
  start = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[0])[0]
  end = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[-1])[1]
  start = millisec(start) #- spacermilli
  end = millisec(end)  #- spacermilli
  print(start, end)
  gidx += 1
  audio[start:end].export(str(gidx) + '.wav', format='wav')

2033 117002
117002 124191
123752 129050
125879 125895
130181 224496
224496 232934
232731 239616
236275 237102
241962 248931
248560 258449


Freeing up some memory

In [ ]:
#del   DEMO_FILE, pipeline, spacer,  audio, dz, newAudio

# Whisper's Transcriptions

Installing Open AI whisper.

**Important:** There is a version conflict with pyannote.audio resulting in an error (see this [RP](https://github.com/pyannote/pyannote-audio/pull/1098)). Our workaround is to first run Pyannote and then whisper. You can safely ignore the error.

In [ ]:
!pip install git+https://github.com/openai/whisper.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-h46ycyj1
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-h46ycyj1
     |████████████████████████████████| 5.5 MB 3.7 MB/s 
     |████████████████████████████████| 7.6 MB 76.6 MB/s 
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175218 sha256=bb61590b3d839fc0639ffac69d3efe05d139508810bfe9067046a0593f28719b
  Stored in directory: /tmp/pip-ephem-wheel-cache-4pop6l_o/wheels/16/15/89/1c7bb31bd0006793a95549d04785121a8a36daad9158e1e43a
Successfully built whisper


Run whisper on all audio files. Whisper generates the transcription and writes it to a file.

In [ ]:
for i in range(gidx+1):
  !whisper {str(i) + '.wav'} --language en --model large

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
Moving 0 files to the new cache system
0it [00:00, ?it/s]
100%|█████████████████████████████████████| 2.87G/2.87G [01:13<00:00, 42.2MiB/s]
tcmalloc: large alloc 3087007744 bytes == 0xa57e000 @  0x7fdbe8cda1e7 0x4b2590 0x5ad01c 0x5dcfef 0x58f92b 0x590c33 0x5e48ac 0x4d20fa 0x51041f 0x58fd37 0x50c4fc 0x5b4ee6 0x58ff2e 0x50d482 0x58fd37 0x50c4fc 0x5b4ee6 0x6005a3 0x607796 0x60785c 0x60a436 0x64db82 0x64dd2e 0x7fdbe88d7c87 0x5b636a
[00:00.000 --> 00:10.940]  So then I come to Cambridge in 1941 as a 17-year-old, and I'd always been interested in physics
[00:10.940 --> 00:15.960]  and applied mathematics of all sorts, and one of the textbooks that I bought as a prize
[00:15.960 --> 00:24.380]  was a textbook in aerodynamics, which I think was because of James Light

# Generating the HTML and/or txt file from the Transcriptions and the Diarization

Reading the transcription file.

In [ ]:
!pip install -U webvtt-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Change or add to the speaker names and collors bellow as you wish `(speaker, textbox color, speaker color)`.

In [ ]:
speakers = {'SPEAKER_00':('Dyson', 'white', 'darkorange'), 'SPEAKER_01':('Interviewer', '#e1ffc7', 'darkgreen') }
def_boxclr = 'white'
def_spkrclr = 'orange'

In the generated HTML,  the transcriptions for each diarization group are written in a box, with the speaker name on the top. By clicking a transcription, the embedded video jumps to the right time .

In [ ]:
preS = '<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <meta http-equiv="X-UA-Compatible" content="ie=edge">\n    <title>' + \
      video_title + \
      '</title>\n    <style>\n        body {\n            font-family: sans-serif;\n            font-size: 18px;\n            color: #111;\n            padding: 0 0 1em 0;\n\t        background-color: #efe7dd;\n        }\n        table {\n             border-spacing: 10px;\n        }\n        th { text-align: left;}\n        .lt {\n          color: inherit;\n          text-decoration: inherit;\n        }\n        .l {\n          color: #050;\n        }\n        .s {\n            display: inline-block;\n        }\n        .c {\n            display: inline-block;\n        }\n        .e {\n            /*background-color: white; Changing background color */\n            border-radius: 20px; /* Making border radius */\n            width: fit-content; /* Making auto-sizable width */\n            height: fit-content; /* Making auto-sizable height */\n            padding: 5px 30px 5px 30px; /* Making space around letters */\n            font-size: 18px; /* Changing font size */\n            display: flex;\n            flex-direction: column;\n            margin-bottom: 10px;\n            white-space: nowrap;\n        }\n\n        .t {\n            display: inline-block;\n        }\n        #player {\n            position: sticky;\n            top: 20px;\n            float: right;\n        }\n    </style>\n\t<script>\n      var tag = document.createElement(\'script\');\n      tag.src = "https://www.youtube.com/iframe_api";\n      var firstScriptTag = document.getElementsByTagName(\'script\')[0];\n      firstScriptTag.parentNode.insertBefore(tag, firstScriptTag);\n      var player;\n      function onYouTubeIframeAPIReady() {\n        player = new YT.Player(\'player\', {\n          //height: \'210\',\n          //width: \'340\',\n          videoId: \'' + \
      video_id + \
      '\',\n        });\n      }\n      function jumptoTime(timepoint, id) {\n        event.preventDefault();\n        history.pushState(null, null, "#"+id);\n        player.seekTo(timepoint);\n        player.playVideo();\n      }\n    </script>\n  </head>\n  <body>\n    <h2>' + \
      video_title + \
      '</h2>\n  <i>Click on a part of the transcription, to jump to its video, and get an anchor to it in the address bar<br><br></i>\n<div  id="player"></div>\n'
postS = '\t</body>\n</html>'

In [ ]:
import webvtt

from datetime import timedelta

html = list(preS)
txt = list("")
gidx = -1
for g in groups:  
  shift = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[0])[0]
  shift = millisec(shift) - spacermilli #the start time in the original video
  shift=max(shift, 0)
  
  gidx += 1
  captions = [[(int)(millisec(caption.start)), (int)(millisec(caption.end)),  caption.text] for caption in webvtt.read(str(gidx) + '.wav.vtt')]
  #captions = (list) webvtt.read(str(gidx) + '.wav.vtt')

  if captions:
    speaker = g[0].split()[-1]
    boxclr = def_boxclr
    spkrclr = def_spkrclr
    if speaker in speakers:
      speaker, boxclr, spkrclr = speakers[speaker] 
    
    html.append(f'<div class="e" style="background-color: {boxclr}">\n');
    html.append(f'<span style="color: {spkrclr}">{speaker}</span><br>\n')
      
    for c in captions:
      start = shift + c[0] 
      start = start / 1000.0   #time resolution ot youtube is Second.
      startStr = '{0:02d}:{1:02d}:{2:06.3f}'.format((int)(start // 3600), 
                                              (int)(start % 3600 // 60), 
                                              start % 60)
      end = shift + c[1] 
      end = end / 1000.0   #time resolution ot youtube is Second.
      endStr = '{0:02d}:{1:02d}:{2:06.3f}'.format((int)(end // 3600), 
                                              (int)(end % 3600 // 60), 
                                              end % 60)            
      
      txt.append(f'[{startStr} --> {endStr}] [{speaker}] {c[2]}\n')
      html.append(f'\t\t\t\t<a href="#{startStr}" id="{startStr}" class="lt" onclick="jumptoTime({int(start)}, this.id)">{c[2]}</a>\n')      

    html.append(f'</div>\n');

html.append(postS)

with open("capspeaker.txt", "w") as file:
  s = "".join(txt)
  file.write(s)
if Source == 'File (Google Drive)':
  print(s)
elif Source == 'Youtube':
  with open("capspeaker.html", "w") as file:    #TODO: proper html embed tag when video/audio from file
    s = "".join(html)
    file.write(s)
    print(s)

<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title></title>
    <style>
        body {
            font-family: sans-serif;
            font-size: 18px;
            color: #111;
            padding: 0 0 1em 0;
	        background-color: #efe7dd;
        }
        table {
             border-spacing: 10px;
        }
        th { text-align: left;}
        .lt {
          color: inherit;
          text-decoration: inherit;
        }
        .l {
          color: #050;
        }
        .s {
            display: inline-block;
        }
        .c {
            display: inline-block;
        }
        .e {
            /*background-color: white; Changing background color */
            border-radius: 20px; /* Making border radius */
            width: fit-content; /* Making auto-sizable width */
            height: fit-content; /* 